# [모듈 4.1] 훈련 스텝 개발

## 0. 기본 세이지 메이커 정보 및 기본 변수 로딩

In [14]:
import boto3
import sagemaker
import pandas as pd

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()

%store -r 
%store

Stored variables and their in-db values:
base_preproc_input_dir                 -> 'opt/ml/processing/input'
dataset_path                           -> 'opt/ml/processing/input/dataset.csv'
default_bucket                         -> 'sagemaker-ap-northeast-2-057716757052'
input_data_uri                         -> 's3://sagemaker-ap-northeast-2-057716757052/fraud2
preprocessing_code_dir                 -> 'fraud/preprocessing.py'
processing_instance_count              -> ParameterInteger(name='ProcessingInstanceCount', p
project_prefix                         -> 'fraud2scratch'
s3_dataset_path                        -> 's3://sagemaker-ap-northeast-2-057716757052/fraud2
test_preproc__dir_artifact             -> 's3://sagemaker-ap-northeast-2-057716757052/sklear
train_model_artifact                   -> 's3://sagemaker-ap-northeast-2-057716757052/fraud2
train_preproc_dir_artifact             -> 's3://sagemaker-ap-northeast-2-057716757052/sklear
val_preproc_dir_artifact               -> 's3:

## 1. 노트북 변수 설정
---

In [15]:
model_prefix = 'fraud2scratch/model'
model_path = f"s3://{default_bucket}/{model_prefix}"

## 2. 모델 빌딩 파이프라인에서  실행 
---



### 모델 빌딩 파이프라인 변수 생성



In [16]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value="ml.m5.xlarge"
)

training_instance_type = ParameterString(
    name="TrainingInstanceType",
    default_value="ml.m5.xlarge"
)

input_data = ParameterString(
    name="InputData",
    default_value=input_data_uri,
)


### 모델 학습을 위한 학습단계 정의 

본 단계에서는 SageMaker의 [XGBoost](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html) 알고리즘을 이용하여 학습을 진행할 것입니다. XGBoost 알고리즘을 이용하도록 Estimator를 구성합니다. 보편적인 학습스크립트를 이용하여 입력 채널에서 정의한 학습데이터를 로드하고, 하이퍼파라미터 설정을 통해 학습을 설정하고, 모델을 학습한 후 `model_dir`경로에 학습된 모델을 저장합니다. 저장된 모델은 이후 호스팅을 위해 사용됩니다. 

학습된 모델이 추출되어 저장될 경로 또한 명시되었습니다. 

`training_instance_type`파라미터가 사용된 것을 확인합니다. 이 값은 본 예제의 파이프라인에서 여러번 사용됩니다. 본 단계에서는 estimator를 선언할 때 전달되었습니다. 


In [17]:
from sagemaker.estimator import Estimator

# XGBoost 알고리즘 컨테이너 URL
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.0-1",
    py_version="py3",
)

xgb_train = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
    output_path=model_path,
    role=role,
)
xgb_train.set_hyperparameters(
    objective="reg:linear",
    num_round=50,
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.7,
    silent=0
)

이전 단계에서 (프로세싱) 전처리 훈련, 검증 데이터 세트를 입력으로 제공 합니다.

In [18]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep

step_train = TrainingStep(
    name="FraudScratchTrain",
    estimator=xgb_train,
    inputs={
        "train": TrainingInput(
            s3_data= train_preproc_dir_artifact,
            content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data= val_preproc_dir_artifact,
            content_type="text/csv"
        )
    },
)

### 모델 빌딩 파이프라인 정의

In [19]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = project_prefix
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type, 
        processing_instance_count,
        training_instance_type,        
        input_data,
    ],
    steps=[step_train],
)

In [20]:
import json

definition = json.loads(pipeline.definition())
# definition

### 파이프라인을 SageMaker에 제출하고 실행하기 

파이프라인 정의를 파이프라인 서비스에 제출합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것입니다.   

In [21]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/fraud2scratch',
 'ResponseMetadata': {'RequestId': '243f04ac-cd9a-49a3-b7fe-add0592c1714',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '243f04ac-cd9a-49a3-b7fe-add0592c1714',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '86',
   'date': 'Tue, 13 Apr 2021 07:55:37 GMT'},
  'RetryAttempts': 0}}

디폴트값을 이용하여 파이프라인을 샐행합니다. 

In [22]:
execution = pipeline.start()

### 파이프라인 운영: 파이프라인 대기 및 실행상태 확인

워크플로우의 실행상황을 살펴봅니다. 

In [23]:
# execution.describe()
execution.wait()

실행이 완료될 때까지 기다립니다.

실행된 단계들을 리스트업합니다. 파이프라인의 단계실행 서비스에 의해 시작되거나 완료된 단계를 보여줍니다.

In [24]:
execution.list_steps()

[{'StepName': 'FraudScratchTrain',
  'StartTime': datetime.datetime(2021, 4, 13, 7, 55, 38, 646000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 4, 13, 7, 58, 54, 600000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:training-job/pipelines-decbp6w0hddy-fraudscratchtrain-pgda1l8u7s'}}}]

### 아티펙트 경로 추출
위의 훈련 스텝이 완료되면 실행해주세요

In [25]:
def get_train_artifact(execution, client, job_type,  kind=0):
    '''
    kind: 0 --> train
    kind: 2 --> test
    '''
    response = execution.list_steps()
    # print("response: ", response)
    proc_arn = response[0]['Metadata'][job_type]['Arn']
    train_job_name = proc_arn.split('/')[-1]
    # print("train_job_name: ", train_job_name)
    response = client.describe_training_job(TrainingJobName = train_job_name)
    # print("\nresponse: ", response)    
    train_model_artifact = response['ModelArtifacts']['S3ModelArtifacts']    
    
    return train_model_artifact

import boto3
client = boto3.client("sagemaker")
    
train_model_artifact = get_train_artifact(execution, client,job_type='TrainingJob', kind=0)
print(" train_model_artifact: ", train_model_artifact)


 train_model_artifact:  s3://sagemaker-ap-northeast-2-057716757052/fraud2scratch/model/pipelines-decbp6w0hddy-FraudScratchTrain-PgDA1l8U7S/output/model.tar.gz


In [27]:
%store train_model_artifact
%store image_uri

Stored 'train_model_artifact' (str)
Stored 'image_uri' (str)
